In [ ]:
# Fist import the library.
import pyrealsense2 as rs
import numpy as np
import cv2
import time, traceback

In [ ]:
# Create a pipeline
pipeline = rs.pipeline()

In [ ]:
# Create a config and configure the pipeline to stream.
# Different resolutions of color and depth streams.

config = rs.config()

In [ ]:
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 60)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 60)

In [ ]:
# Start streaming
profile = pipeline.start(config)

In [ ]:
align_to = rs.stream.color
align = rs.align(align_to)

In [ ]:
frames = pipeline.wait_for_frames()

### Dhash

In [ ]:
def cal_diff(image, hash_size=8):
    width = 9
    height = 8
    # Reshape image to 9*8 and grey it.
    image = Image.fromarray(image)
    img = image.resize((width, height))
    gray_image = img.convert('L')
    pixels = list(gray_image.getdata())
    difference = []
    for h in range(height):
        start_index = h * width
        for w in range(width - 1):
            left_pixel_index = start_index + w
            difference.append(pixels[left_pixel_index] > pixels[left_pixel_index + 1])
    return difference

def hamming_distance(first, second):
    
    hamming_distance = 0
    first_diff = cal_diff(first)
    second_diff = cal_diff(second)
    
    for index, img_1 in enumerate(first_diff):
        img_2 = second_diff[index]
        if img_1 != img_2:
            hamming_distance += 1
    return hamming_distance

### Average Hash

In [ ]:
def average_hash(image):
    """
    :param image: numpy array or img.
    :return hash_string: str
    """
    image = cv2.resize(image,(8,8),interpolation=cv2.INTER_CUBIC)
    gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    s = 0
    hash_string = ''
    
    for i in range(8):
        for j in range(8):
            s = s + gray[i, j]
    avg = s/64
    
    for i in range(8):
        for j in range(8):
            if gray[i, j] > avg:
                hash_string += '1'
            else:
                hash_string += '0'
    return hash_string

def hamming_distance_with_hash(dhash1, dhash2):
    """
    *Private method*
    根据dHash值计算hamming distance
    :param dhash1: str
    :param dhash2: str
    :return: 汉明距离(int)
    """
    difference = (int(dhash1, 16)) ^ (int(dhash2, 16))
    return bin(difference).count("1")

In [ ]:
background = None

try:
    while True:
        frames = pipeline.wait_for_frames()
        
        aligned_frames = align.process(frames)
        aligened_depth_frame = aligned_frames.get_depth_frame()
        color_frame = aligned_frames.get_color_frame()
        
        if not aligened_depth_frame or not color_frame:
            continue
            
        depth_image = np.asanyarray(aligened_depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())
        
        if background is None:
            background = color_image
            continue
        distance = hamming_distance(background, color_image)
        print("[Distance]: %s" % distance)
        
        background = color_image
        
        if distance > 5:
            #cv2.imwrite("/home/commaai-03/image/depth/%.9f.pgm" % time.time(), depth_image)
            #cv2.imwrite("/home/commaai-03/image/color/%.9f.jpg" % time.time(), color_image)
            print("Shape:  ", depth_image.shape)
        key = cv2.waitKey(1)
        if key & 0xFF == ord('q') or key == 27:
            cv2.destroyAllWindows()
            break
except Exception as e:
    traceback.print_exc()